In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='10GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://140.113.67.110:8787/status,
Dashboard: http://140.113.67.110:8787/status,Workers: 1
Total threads: 4,Total memory: 9.31 GiB
Status: running,Using processes: False
Comm: inproc://140.113.67.110/439806/1,Workers: 1
Dashboard: http://140.113.67.110:8787/status,Total threads: 4
Started: Just now,Total memory: 9.31 GiB
Comm: inproc://140.113.67.110/439806/4,Total threads: 4
Dashboard: http://140.113.67.110:33119/status,Memory: 9.31 GiB
Nanny: None,


In [5]:
types = dict(
    vendor_name='object',
    Trip_Pickup_DateTime='object', 
    Trip_Dropoff_DateTime='object', 
    Passenger_Count='int64',
    Trip_Distance='float64',
    Start_Lon='float64',
    Start_Lat='float64',
    Rate_Code='float64',
    store_and_forward='float64',
    End_Lon='float64',
    End_Lat='float64',
    Payment_Type='object', 
    Fare_Amt='float64',
    surcharge='float64',
    mta_tax='float64',
    Tip_Amt='float64',
    Tolls_Amt='float64',
    Total_Amt='float64',
)
df = dd.read_csv('yellow_tripdata_2009-0*.csv', dtype=types)
df = df[['Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat', 'Trip_Pickup_DateTime', 'Passenger_Count']]


In [3]:
sub_df = df.dropna(subset=['Start_Lon', 'Start_Lat'])[['Start_Lon', 'Start_Lat']]
# sub_df = sub_df.loc[sub_df['Start_Lon'] < -70].loc[sub_df['Start_Lat'] > 35]

sub_df = sub_df[['Start_Lon', 'Start_Lat']].values.compute()

unique, counts = np.unique(sub_df, axis=0, return_counts=True)
count_sort_ind = np.argsort(-counts)
unique = unique[count_sort_ind,:]
counts = counts[count_sort_ind]

print('\n'.join(
    [f'{u}: {c}' for u, c in zip(unique[:5], counts[:5])]
))

del sub_df, unique, counts, count_sort_ind

[0. 0.]: 605041
[-73.137393  41.366138]: 54807
[-73.951818  40.733697]: 5197
[-73.988537  40.698458]: 2286
[-73.989039  40.75808 ]: 1336
[-73.945795  40.778752]: 1331
[-73.989054  40.758087]: 1136
[-73.937513  40.758152]: 1042
[-73.952003  40.73376 ]: 859
[-73.991085  40.73331 ]: 827
[-73.989048  40.758084]: 774
[-73.844311  40.721355]: 680
[-73.942413  40.749338]: 652
[-73.951413  40.748827]: 645
[-73.921045  40.773093]: 597
[-73.987343  40.759963]: 560
[-73.933672  40.752364]: 510
[-73.848378  40.738013]: 386
[-73.978493  40.68542 ]: 368
[-73.96037   40.581487]: 349


In [4]:
sub_df = df.dropna(subset=['End_Lon', 'End_Lat'])[['End_Lon', 'End_Lat']]
# sub_df = sub_df.loc[sub_df['End_Lon'] < -70].loc[sub_df['End_Lat'] > 35]
# sub_df = client.persist(sub_df)
sub_df = sub_df[['End_Lon', 'End_Lat']].values.compute()

aaaa = np.round(sub_df, 6).copy()

unique, counts = np.unique(aaaa, axis=0, return_counts=True)
count_sort_ind = np.argsort(-counts)
unique = unique[count_sort_ind,:]
counts = counts[count_sort_ind]

print('\n'.join(
    [f'{u}: {c}' for u, c in zip(unique[:5], counts[:5])]
))

del sub_df, unique, counts, count_sort_ind

[0. 0.]: 593865
[-73.137393  41.366138]: 45987
[-73.951818  40.733697]: 5197
[-73.988537  40.698458]: 2286
[-73.989039  40.75808 ]: 1336
[-73.945795  40.778752]: 1331
[-73.989054  40.758087]: 1136
[-73.937513  40.758152]: 1044
[-73.952003  40.73376 ]: 859
[-73.991085  40.73331 ]: 838
[-73.989048  40.758084]: 774
[-73.942413  40.749338]: 652
[-73.951413  40.748827]: 650
[-73.921045  40.773093]: 597
[-73.987343  40.759963]: 559
[-73.933672  40.752364]: 510
[-73.96037   40.581487]: 406
[-73.848378  40.738013]: 385
[-73.978493  40.68542 ]: 369
[-73.953833  40.813287]: 342


In [5]:
sub_df = df['Trip_Pickup_DateTime'].str[11:13]
hours = sub_df.values.compute()

unique, counts = np.unique(hours, return_counts=True)
print(max(counts), unique[np.argmax(counts)])
print(min(counts), unique[np.argmin(counts)])

del sub_df, hours

2769506 19
379852 05


In [6]:
Passenger_Count = df['Passenger_Count']
Passenger_Count = Passenger_Count.values.compute()
unique, counts = np.unique(Passenger_Count, return_counts=True)

print(np.asarray((unique, counts)).T)

del Passenger_Count, unique, counts

[[       0      882]
 [       1 28512760]
 [       2  6775362]
 [       3  1822286]
 [       4   845145]
 [       5  3715461]
 [       6   187760]
 [       7        1]
 [      13        1]
 [      17        1]
 [     113        1]
 [     129        2]
 [     208      244]]


In [7]:
sub_df = df.dropna(subset=['Passenger_Count', 'Start_Lon', 'Start_Lat'])[['Passenger_Count', 'Start_Lon', 'Start_Lat']]
big = sub_df[sub_df['Passenger_Count'] >1]

big = big[['Start_Lon', 'Start_Lat']].values.compute()

unique, counts = np.unique(big, axis=0, return_counts=True)
count_sort_ind = np.argsort(-counts)
unique = unique[count_sort_ind,:]
counts = counts[count_sort_ind]

big_unique, big_counts = unique[:5], counts[:5]
del unique, counts, count_sort_ind, big

small = sub_df[sub_df['Passenger_Count'] <= 1]

small = small[['Start_Lon', 'Start_Lat']].values.compute()

unique, counts = np.unique(small, axis=0, return_counts=True)
count_sort_ind = np.argsort(-counts)
unique = unique[count_sort_ind,:]
counts = counts[count_sort_ind]

small_unique, small_counts = unique[:5], counts[:5]
del sub_df, unique, counts, count_sort_ind, small

print('\n'.join(
    [f'{u}: {c}' for u, c in zip(big_unique, big_counts)]
))

print('\n'.join(
    [f'{u}: {c}' for u, c in zip(small_unique, small_counts)]
))

[0. 0.]: 157983
[-73.137393  41.366138]: 13568
[-73.951818  40.733697]: 941
[-73.991085  40.73331 ]: 805
[-73.937513  40.758152]: 729
[0. 0.]: 447058
[-73.137393  41.366138]: 41239
[-73.951818  40.733697]: 4256
[-73.988537  40.698458]: 1637
[-73.945795  40.778752]: 1016


In [8]:
sub_df = df.dropna(subset=['Passenger_Count', 'End_Lon', 'End_Lat'])[['Passenger_Count', 'End_Lon', 'End_Lat']]
big = sub_df[sub_df['Passenger_Count'] > 1]

# big = big.loc[big['End_Lon'] < -70].loc[big['End_Lat'] > 35]
big = big[['End_Lon', 'End_Lat']].values.compute()

unique, counts = np.unique(big, axis=0, return_counts=True)
count_sort_ind = np.argsort(-counts)

unique = unique[count_sort_ind,:]
counts = counts[count_sort_ind]

big_unique, big_counts = unique[:5], counts[:5]
del unique, counts, count_sort_ind, big

small = sub_df[sub_df['Passenger_Count'] <= 1]

small = small[['End_Lon', 'End_Lat']].values.compute()

unique, counts = np.unique(small, axis=0, return_counts=True)
count_sort_ind = np.argsort(-counts)
unique = unique[count_sort_ind,:]
counts = counts[count_sort_ind]

small_unique, small_counts = unique[:5], counts[:5]
del sub_df, unique, counts, count_sort_ind, small

print('\n'.join(
    [f'{u}: {c}' for u, c in zip(big_unique, big_counts)]
))

print('\n'.join(
    [f'{u}: {c}' for u, c in zip(small_unique, small_counts)]
))

[0. 0.]: 153889
[-73.137393  41.366138]: 11167
[-73.951818  40.733697]: 941
[-73.991085  40.73331 ]: 815
[-73.937513  40.758152]: 730
[0. 0.]: 439976
[-73.137393  41.366138]: 34820
[-73.951818  40.733697]: 4256
[-73.988537  40.698458]: 1637
[-73.945795  40.778752]: 1016


In [9]:
sub_df = df.dropna(subset=['Passenger_Count', 'Trip_Pickup_DateTime'])[['Passenger_Count', 'Trip_Pickup_DateTime']]

big = sub_df[sub_df['Passenger_Count'] > 1]
big = big['Trip_Pickup_DateTime'].str[11:13]
hours = big.values.compute()

big_unique, big_counts = np.unique(hours, return_counts=True)

del big, hours

small = sub_df[sub_df['Passenger_Count'] <= 1]
small = small['Trip_Pickup_DateTime'].str[11:13]
hours = small.values.compute()

small_unique, small_counts = np.unique(hours, return_counts=True)

del sub_df, small, hours

print(max(big_counts), big_unique[np.argmax(big_counts)])
print(min(big_counts), big_unique[np.argmin(big_counts)])
print(max(small_counts), small_unique[np.argmax(small_counts)])
print(min(small_counts), small_unique[np.argmin(small_counts)])

942712 19
95109 05
1826794 19
284743 05
